In [ ]:
%load_ext autoreload
%autoreload 2
%aimport
# %reload_ext autoreload

import sys 
from os.path import abspath
paths = [abspath('../..'), "/home/imendoza/alcca/nbody-relaxed/packages/minnow"]
print(paths)

for path in paths: 
    if path not in sys.path: 
        sys.path.insert(0, path)

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import re 
from astropy.table import Table
import astropy.table
import json
from scipy import stats

from relaxed import halo_parameters, halo_catalogs 
import warnings

# Read catalog

In [ ]:
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
names = ['m11', 'm12']
params = ['cvir', 'x0', 'v0', 't/|u|']

fig, axes = plt.subplots(2,2,figsize=(10,10))
axes = axes.flatten()
for i, name in enumerate(names):
    output = f'../../temp/output_{name}/'
    cat_file = Path(output, 'final_table.csv')
    hcat = halo_catalogs.HaloCatalog('Bolshoi', cat_file, label=name)
    hcat.load_cat_csv()
    
    z_map_file = Path(output, 'z_map.json')
    with open(z_map_file,'r') as fp: 
        scale_map = json.load(fp) # map from i -> sca    
    indices = np.array(list(scale_map.keys()))
    scales = np.array(list(scale_map.values()))
    keep = scales>.15 
    indices = indices[keep]
    scales = scales[keep]
    for i, param in enumerate(params):
        ax = axes[i]

        corrs = []
        for i in indices:
            colname = f'mvir_a{i}'
            nankeep = ~np.isnan(hcat.cat[colname])

            # get mass fraction at this scale
            mvir = hcat.cat['mvir'][nankeep]
            ms = hcat.cat[colname][nankeep]
            ms = ms / mvir
            pvalue = hcat.cat[param][nankeep]

            # get correlation.
            assert np.all(ms>0) and np.all(~np.isnan(ms))
            assert np.all(mvir>0)
            assert np.all(pvalue>0)
            corrs.append(stats.spearmanr(ms, pvalue)[0])

        corrs = np.array(corrs)    
        ax.plot(scales, corrs, label=name)
        ax.set_ylim(-0.9, 0.9)
        ax.set_ylabel(f"$\\rho({param}, m(a))$", size=18)
        ax.set_xlabel(f"$a$", size=18)
plt.legend()
plt.tight_layout()
        
    